In [ ]:
import emoji
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM
from keras.callbacks import EarlyStopping,ModelCheckpoint

In [ ]:
train_dataset=pd.read_csv('/home/srinivasan/Downloads/twitter-emoji-prediction/Train.csv')
test_dataset=pd.read_csv('/home/srinivasan/Downloads/twitter-emoji-prediction/Test.csv')

In [ ]:
x_train=train_dataset.iloc[:,1].values
y_train=train_dataset.iloc[:,2].values
x_test=test_dataset.iloc[:,2].values
y_train=to_categorical(y_train,num_classes=20)

In [ ]:
emoji_dictionary={"0"  :    ":winking_face_with_tongue:",
                  "1"  :    ":camera_with_flash:",
                  "2"  :    ":smiling_face_with_heart-eyes:",
                  "3"  :    ":face_with_tears_of_joy:",
                  "4"  :    ":winking_face:",
                  "5"  :    ":Christmas_tree:",
                  "6"  :    ":camera:",
                  "7"  :    ":fire:",
                  "8"  :    ":face_blowing_a_kiss:",
                  "9"  :    ":black_heart:",
                  "10" :    ":beaming_face_with_smiling_eyes:",
                  "11" :    ":United_States:",
                  "12" :    ":eight-pointed_star:",
                  "13" :    ":sparkles:",
                  "14" :    ":blue_heart:",
                  "15" :    ":two_hearts:",
                  "16" :    ":smiling_face_with_sunglasses:",
                  "17" :    ":smiling_face_with_smiling_eyes:",
                  "18" :    ":white_heart:",
                  "19" :    ":hundred_points:"
                }

In [ ]:
f=open('/home/srinivasan/Downloads/glove.6B.50d.txt',encoding='utf-8')
embeddings_index={}

for line in f:
    values=line.split()
    words=values[0]
    coeffs=np.asarray(values[1:],dtype='float')
    embeddings_index[word]=coeffs
f.close()   

In [ ]:
def embedding_output(X):
    max_len=30
    emb_dim=50
    embedding_out=np.zeros((X.shape[0],max_len,emb_dim))
    
    for i in range(X.shape[0]):
        X[i]=X[i].split()
        for j in range(len(X[i])):
            embedding_out[i][j]=embeddings_index[X[i][j].lower()] 
    return embedding_out

In [ ]:
embeddings_matrix_train=embedding_output(x_train)
embeddings_matrix_test=embedding_output(x_test)

In [ ]:
model=Sequential()
model.add(LSTM(64,input_shape=(30,50)),return_sequences=True) 
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(units=20,activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
checkpoint=ModelCheckpoint("best_model.h5",monitor='loss',verbose=True,save_best_only=True)

In [ ]:
earlystopping=EarlyStopping(patience=3)

In [ ]:
hist=model.fit(embeddings_matrix_train,y_train,epochs=20,batch_size=64,shuffle=True)

In [ ]:
pred=model.predict(embeddings_matrix_test)